## <a name="Wheat Detection">Plant Pathology 2021 FGVC8 </a>

#### <a name="About_Competition"> Giới thiệu </a>

Táo là một trong những loại cây ăn quả ôn đới quan trọng nhất trên thế giới. Bệnh cháy lá  là mối đe dọa lớn đối với năng suất và chất lượng chung của vườn táo. Quy trình chẩn đoán bệnh trên vườn táo hiện nay dựa trên việc dò tìm thủ công của con người, tốn nhiều thời gian và chi phí.

Mặc dù các mô hình dựa trên thị giác máy tính đã cho thấy nhiều hứa hẹn trong việc xác định bệnh thực vật, nhưng vẫn còn một số hạn chế cần được giải quyết. Sự khác biệt lớn về các triệu chứng hình ảnh của một bệnh đơn lẻ trên các giống táo khác nhau, hoặc các giống mới có nguồn gốc được trồng trọt, là những thách thức lớn đối với việc xác định bệnh dựa trên thị giác máy tính. Những biến thể này phát sinh do sự khác biệt trong môi trường chụp ảnh và tự nhiên, ví dụ, màu sắc lá và hình thái lá, tuổi của các mô bị nhiễm bệnh, nền ảnh không đồng nhất và độ chiếu sáng khác nhau trong quá trình chụp ảnh, v.v.

Plant Pathology 2021-FGVC8 có tập dữ liệu thí điểm gồm 3.651 hình ảnh RGB về bệnh lá trên quả táo.Tập dữ liệu chứa khoảng 23.000 hình ảnh RGB chất lượng cao về các bệnh trên lá táo, bao gồm một tập dữ liệu lớn về bệnh được chuyên gia chú thích. Bộ dữ liệu này phản ánh các tình huống thực tế bằng cách thể hiện các nền không đồng nhất của hình ảnh chiếc lá được chụp ở các giai đoạn trưởng thành khác nhau và vào các thời điểm khác nhau trong ngày trong các cài đặt máy ảnh tiêu cự khác nhau.
                           

#### <a name="Specific Objectives">Xác định mục tiêu</a>           

Mục tiêu chính của cuộc thi là phát triển các mô hình dựa trên máy học để phân loại chính xác một hình ảnh lá nhất định từ bộ dữ liệu thử nghiệm cho một loại bệnh cụ thể và xác định một bệnh riêng lẻ từ nhiều triệu chứng bệnh trên một hình ảnh lá đơn lẻ.


#### <a name="Yêu cầu">Yêu cầu</a>           

Mục tiêu chính của cuộc thi là phát triển các mô hình dựa trên máy học để phân loại chính xác một hình ảnh lá nhất định từ bộ dữ liệu thử nghiệm cho một loại bệnh cụ thể và xác định một bệnh riêng lẻ từ nhiều triệu chứng bệnh trên một hình ảnh lá đơn lẻ.


#### <a name="dataset_description">Mô tả dữ liệu</a>: 

Dữ liệu lưu giữ hình ảnh của cây táo. Lá cây khỏe mạnh và bị nhiễm bệnh.

Files train.csv - dữ liệu tập huấn luyện.

Image - ID của hình ảnh

Label - các lớp mục tiêu thể hiện tất cả các bệnh được tìm thấy trong hình ảnh. Những lá không tốt có quá nhiều bệnh để phân loại bằng mắt thường sẽ có lớp phức tạp, và cũng có thể có một tập hợp con của các bệnh được xác định.


sample_submission.csv - Tệp gửi mẫu ở định dạng:

    1. image
    2. labels

train_images - tập huấn luyện.
test_images - bộ thử nghiệm. Cuộc thi này có một bộ thử nghiệm ẩn: chỉ có ba hình ảnh được cung cấp ở đây dưới dạng mẫu trong khi 5.000 hình ảnh còn lại sẽ có sẵn trong sổ ghi chép sau khi nó được gửi.

Phân loại Labels:
*     healthy
*     complex
*     frog_eye_leaf_spot
*     frog_eye_leaf_spot complex
*     powdery_mildew
*     powdery_mildew complex
*     rust
*     rust complex
*     rust frog_eye_leaf_spot
*     scab
*     scab frog_eye_leaf_spot
*     scab frog_eye_leaf_spot complex


# Nội dung

* [<font size=4>EDA</font>](#1)
    * [Chuẩn bị dữ liệu](#1.1)
    * [Một số ảnh ví dụ từ tập dữ liệu](#1.2)
    * [Phân phối RBG](#1.3)
    * [Parallel categories plot](#1.4)


## Importing các thư viện cần thiết

In [ ]:
import os
from tqdm import tqdm

# Data Processing Libraries 

import pandas as pd 
import numpy as np 


# Feature Engineering Libraries

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

# Data Visualisation libraries 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import cv2
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

## Image Augmentation 

# skimage
from skimage.io import imshow, imread, imsave
from skimage.transform import rotate, AffineTransform, warp,rescale, resize, downscale_local_mean
from skimage import color,data
from skimage.exposure import adjust_gamma
from skimage.util import random_noise


# 3D scatter plot
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors


#OpenCV-Python
import cv2

# imgaug
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

# Albumentations
import albumentations as A

SAMPLE_LEN=100

# Chuẩn bị dữ liệu

In [ ]:
train_image_path = '../input/plant-pathology-2021-fgvc8/train_images'
test_image_path = '../input/plant-pathology-2021-fgvc8/test_images'
train_df_path = '../input/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'

In [ ]:
#Đọc dữ liệu
df_train = pd.read_csv(train_df_path)

In [ ]:
#In dữ liệu
df_train.head()

In [ ]:
#Kích thước dữ liệu
df_train.shape




<!-- #### <a>Đếm số lượng các labels</a>            -->
### Đếm số lượng các labels

In [ ]:
#Số lượng của mỗi label
df_train['labels'].value_counts()

In [ ]:
# sns.histplot(df_train['labels'].value_counts(sort=True))

### Lập biểu đồ

In [ ]:
plt.figure(figsize=(15,12))
labels = sns.barplot(df_train.labels.value_counts().index,df_train.labels.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)

In [ ]:
source = df_train['labels'].value_counts()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
fig.update_layout(title='Label distribution')
fig.show()

## Kết Luận

- Tập dữ liệu khá không cân bằng theo biểu đồ hình tròn ở trên
- Chúng tôi sẽ chọn chiến lược lấy mẫu thích hợp để giải quyết vấn đề này (sử dụng albumentation)

# Một số ảnh ví dụ từ tập dữ liệu

Chúng tôi sẽ kiểm tra kích thước của 300 hình ảnh đầu tiên

Như bạn có thể thấy bên dưới, tất cả các hình ảnh có kích thước khác nhau.

In [ ]:
img_shapes = {}
for image_name in tqdm(os.listdir(train_image_path)[:300]):
    image = cv2.imread(os.path.join(train_image_path, image_name))
    img_shapes[image.shape] = img_shapes.get(image.shape, 0) + 1

print(img_shapes)

In [ ]:
def visualize_batch(path,image_ids, labels):
    plt.figure(figsize=(16, 12))
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join(path, image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"Class: {label}", fontsize=12)
        plt.axis("off")
    plt.show()

In [ ]:
tmp_df = df_train.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values
visualize_batch(train_image_path,image_ids,labels)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
  
# Label encoding.
df_train["labels_code"]= label_encoder.fit_transform(df_train[["labels"]])
df_train

In [ ]:
#label = complex
tmp_df = df_train[df_train["labels_code"] == 0]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = frog_eye_leaf_spot
tmp_df = df_train[df_train["labels_code"] == 1]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = frog_eye_leaf_spot complex
tmp_df = df_train[df_train["labels_code"] == 2]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = healthy
tmp_df = df_train[df_train["labels_code"] == 3]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = powdery_mildew
tmp_df = df_train[df_train["labels_code"] == 4]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = powdery_mildew complex
tmp_df = df_train[df_train["labels_code"] == 5]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = rust
tmp_df = df_train[df_train["labels_code"] == 6]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = rust complex
tmp_df = df_train[df_train["labels_code"] == 7]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = rust frog_eye_leaf_spot
tmp_df = df_train[df_train["labels_code"] == 8]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = scab
tmp_df = df_train[df_train["labels_code"] == 9]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = scab frog_eye_leaf_spot
tmp_df = df_train[df_train["labels_code"] == 10]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

In [ ]:
#label = scab frog_eye_leaf_spot complex
tmp_df = df_train[df_train["labels_code"] == 11]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image"].values
labels = tmp_df["labels"].values

visualize_batch(train_image_path, image_ids, labels)

Chúng đã plot một vài hình ảnh trong training data ở trên (các giá trị RGB có thể được nhìn thấy bằng cách di chuột qua hình ảnh). Các phần màu xanh lá cây của hình ảnh có giá trị màu xanh lam rất thấp, nhưng ngược lại, các phần màu nâu có giá trị màu xanh lam cao. Điều này cho thấy rằng các phần màu xanh lá cây (healthy) của hình ảnh có giá trị màu xanh lam thấp, trong khi các phần unhealthy có nhiều khả năng có giá trị màu xanh lam cao. 
**Điều này có thể cho thấy rằng kênh màu xanh lam có thể là chìa khóa để phát hiện bệnh trên cây trồng**

In [ ]:
def load_image(image_id):
    file_path = image_id
    image = cv2.imread(train_image_path+'/'+ file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Just take 100 sample images with SAMPLE_LEN=100 for RBG Channel Analysis

train_images = df_train["image"][:SAMPLE_LEN].apply(load_image)

In [ ]:
red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

# Phân phối RBG (Tất cả các giá trị kênh)

Histofram là một biểu diễn đồ họa cho biết tần suất xuất hiện của các giá trị màu khác nhau trong hình ảnh. Trong không gian màu RGB, các giá trị pixel nằm trong khoảng từ 0 đến 255 trong đó 0 là màu đen và 255 là màu trắng. Phân tích biểu đồ có thể giúp chúng ta hiểu được phân bố độ sáng, độ tương phản và cường độ của hình ảnh. Bây giờ chúng ta hãy xem biểu đồ của một mẫu được chọn ngẫu nhiên từ mỗi danh mục.

# Phân phối Kênh Đỏ 

In [ ]:
fig = ff.create_distplot([red_values], group_labels=["R"], colors=["red"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Phân phối Kênh Đỏ")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

### Quan sát :
Các giá trị kênh màu đỏ có vẻ gần như phân phối chuẩn, nhưng hơi lệch về bên trái (Độ lệch âm). Điều này cho thấy rằng kênh màu đỏ có xu hướng tập trung nhiều hơn ở các giá trị cao hơn, vào khoảng 100. Có sự thay đổi lớn về giá trị màu đỏ trung bình trên các hình ảnh.

In [ ]:
fig = ff.create_distplot([green_values], group_labels=["G"], colors=["green"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Phân phối Kênh Xanh Lá")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

### Quan sát: 
Giá trị kênh màu xanh lá cây có phân phối đồng đều hơn giá trị kênh màu đỏ nhưng lệch phải, với đỉnh nhỏ hơn. Sự phân bố cũng có độ lệch bên phải (trái ngược với màu đỏ) và chế độ lớn hơn khoảng 160. Điều này cho thấy rằng màu xanh lá cây rõ nét hơn trong những hình ảnh này so với màu đỏ, điều này có ý nghĩa, bởi vì đây là hình ảnh của những chiếc lá!

# Distribution of Blue Channel Values

In [ ]:
fig = ff.create_distplot([blue_values], group_labels=["B"], colors=["blue"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Phân phối Kênh Xanh Lam")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

### Quan sát: 

Kênh màu xanh lam có sự phân bố đồng đều nhất trong số ba kênh màu, với độ lệch tối thiểu (lệch một chút sang trái). Kênh màu xanh lam cho thấy sự thay đổi lớn giữa các hình ảnh trong tập dữ liệu.

# Tất cả các kênh hợp lại

In [ ]:
fig = go.Figure()

for idx, values in enumerate([red_values, green_values, blue_values]):
    if idx == 0:
        color = "Red"
    if idx == 1:
        color = "Green"
    if idx == 2:
        color = "Blue"
    fig.add_trace(go.Box(x=[color]*len(values), y=values, name=color, marker=dict(color=color.lower())))
    
fig.update_layout(yaxis_title="Mean value", xaxis_title="Color channel",
                  title="Mean value vs. Color channel", template="plotly_white")

In [ ]:
fig = ff.create_distplot([red_values, green_values, blue_values],
                         group_labels=["R", "G", "B"],
                         colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution of red channel values", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[2].marker.line.color = 'rgb(0, 0, 0)'
fig.data[2].marker.line.width = 0.5
fig

In [ ]:
image = train_images[10]
imshow(image)
print(image.shape)

## 3D scatter plot cho ảnh với hệ màu RGB


In [ ]:
r, g, b = cv2.split(image)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")

pixel_colors = image.reshape((np.shape(image)[0]*np.shape(image)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.show()

## 3D scatter plot cho ảnh với hệ màu HSV

In [ ]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
h, s, v = cv2.split(hsv_image)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")

axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.show()

# Parallel categories plot

In [ ]:
df_train['label_list'] = df_train['labels'].str.split(' ')

Distinct List of labels 



*     healthy
*     complex
*     rust
*     frog_eye_leaf_spot
*     powdery_mildew
*     scab

In [ ]:
lbls = ['healthy','complex','rust','frog_eye_leaf_spot','powdery_mildew','scab']
for x in lbls:
    df_train[x]=0

In [ ]:
def lbl_lgc(col,lbl_list):
    if col in lbl_list:
        res = 1 
    else:
        res = 0
    return res

In [ ]:
lbls = ['healthy','complex','rust','frog_eye_leaf_spot','powdery_mildew','scab']

for x in lbls:
    df_train[x] = np.vectorize(lbl_lgc)(x,df_train['label_list'])

In [ ]:
df_train

In [ ]:
df_train_lbl_onehot = pd.get_dummies(df_train, columns=["labels"], prefix=["LBL"])

In [ ]:
df_train_lbl_onehot.columns

In [ ]:
plt.figure(figsize=(35,20))
fig = px.parallel_categories(df_train[['healthy','complex','rust','frog_eye_leaf_spot','powdery_mildew','scab']], color="healthy", color_continuous_scale="sunset",\
                             title="Parallel categories plot of targets")
fig

### Quan sát: 

Trong sơ đồ trên, chúng ta có thể thấy mối quan hệ giữa tất cả 6 loại. Đúng như dự đoán, không thể nào một chiếc lá khỏe mạnh lại có thể bị vảy, gỉ sắt, hay nhiều bệnh. Ngoài ra, mỗi chiếc lá không khỏe mạnh đều có một trong các bệnh vảy, gỉ sắt hoặc nhiều bệnh. Tần suất của mỗi kết hợp có thể được nhìn thấy bằng cách di chuột qua plot.